In [2]:
import pandas as pd
import numpy as np

### Warnings
import warnings

### Text Preprocessing and Natural Language Processing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import nltk
import re
import spacy
from wordcloud import WordCloud

# For parallel processing
import multiprocessing

import torch.nn as nn
import torch
import torch.nn.functional as F

In [3]:
df = pd.read_csv("Merged_df.csv")

C:\Users\Wade\AppData\Local\Temp\ipykernel_20628\1427703504.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Merged_df.csv")


In [4]:
df.head(2)

Unnamed: 0  UserID  MovieID  Rating  Timestamp  \
0           0       1     1193       5  978300760   
1           1       1      661       3  978302109   

                                    Title                        Genres  \
0  One Flew Over the Cuckoo's Nest (1975)                         Drama   
1        James and the Giant Peach (1996)  Animation|Children's|Musical   

  Gender  Age  Occupation  ...  \
0      F    1          10  ...   
1      F    1          10  ...   

                                            overview popularity  \
0  A petty criminal fakes insanity to serve his s...     36.673   
1  When the young orphan boy James spills a magic...     27.497   

                        poster_path                                   tagline  \
0  /3jcbDmRFiQ83drXNOvRDeKHxS0C.jpg   If he's crazy, what does that make you?   
1  /nl2oB6EbD1fHFuP2TLUHDtqs7Ux.jpg  Adventures this big don't grow on trees.   

                                  genres  \
0                                  Drama   
1  Adventure, Animation, Family, Fantasy   

                      production_companies      production_countries  \
0                            Fantasy Films  United States of America   
1  Allied Filmmakers, Walt Disney Pictures  United States of America   

  spoken_languages    year                              title_year  
0          English  1975.0  One Flew Over the Cuckoo's Nest (1975)  
1          English  1996.0        James and the Giant Peach (1996)  

[2 rows x 38 columns]

In [5]:
## Pivot the table to obtain user item similarity matrix
from sklearn.metrics import mean_squared_error

ratings_matrix = pd.pivot_table(df.copy(), values='Rating', index='UserID', columns='MovieID')
ratings_matrix.head(2)

MovieID  1     2     3     4     5     6     7     8     10    11    ...  \
UserID                                                               ...   
1         5.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
2         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

MovieID  3943  3944  3945  3946  3947  3948  3949  3950  3951  3952  
UserID                                                               
1         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[2 rows x 3672 columns]

In [22]:
# uncomment to run and save the similarity matrix for the further calculations below
# the similarity matrix is calculating for each user and his similarity to other users based on their rating scores
# similarity_matrix = ratings_matrix.T.corr()

In [7]:
# similarity_matrix.to_csv("sim_matrix.csv")

In [8]:
similarity_matrix = pd.read_csv("sim_matrix.csv", dtype={"UserID": int})
similarity_matrix.set_index('UserID', inplace=True)
similarity_matrix.index =  similarity_matrix.index.astype(int)
similarity_matrix.head(2)

1         2         3         4         5         6         7  \
UserID                                                                         
1       1.000000  0.416667 -0.332182  0.333333 -0.172516  0.099340       NaN   
2       0.416667  1.000000  0.236834 -0.270031 -0.208333 -0.061859  0.154303   

               8         9        10  ...      6031      6032      6033  6034  \
UserID                                ...                                       
1      -0.583333  0.330572 -0.069171  ... -0.160128  0.755929  1.000000   NaN   
2       0.024968  0.205112 -0.054568  ... -0.709299  0.258341  0.368394   NaN   

            6035      6036      6037  6038      6039      6040  
UserID                                                          
1       0.381246 -0.248047 -0.451754   NaN  0.183804 -0.043519  
2       0.390907  0.363309  0.308145  -0.5  0.572078 -0.012620  

[2 rows x 6040 columns]

Splitting data for testing how accurate similarity matrix can predict back the user ratings

In [9]:

# Group the DataFrame by user
grouped = df.copy().groupby('UserID', group_keys=False)

# Sample 20% of each user's ratings
test_df = grouped.apply(lambda x: x.sample(frac=0.005))
test_df.reset_index(drop=True, inplace=True)
test_df.head(2)

C:\Users\Wade\AppData\Local\Temp\ipykernel_20628\2758450849.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df = grouped.apply(lambda x: x.sample(frac=0.01))


Unnamed: 0  UserID  MovieID  Rating  Timestamp  \
0          51       1      608       4  978301398   
1         123       2     1372       3  978299941   

                                           Title                   Genres  \
0                                   Fargo (1996)     Crime|Drama|Thriller   
1  Star Trek VI: The Undiscovered Country (1991)  Action|Adventure|Sci-Fi   

  Gender  Age  Occupation  ...  \
0      F    1          10  ...   
1      M   56          16  ...   

                                            overview popularity  \
0  Jerry, a small-town Minnesota car salesman is ...     22.433   
1  After years of war, the Federation and the Kli...     29.826   

                        poster_path  \
0  /rt7cpEr1uP6RTZykBFhBTcRaKvG.jpg   
1  /tvTOJD7Gz668GLy2nNdLRQvpPsv.jpg   

                                      tagline  \
0  A lot can happen in the middle of nowhere.   
1             The battle for peace has begun.   

                                         genres  \
0                        Crime, Drama, Thriller   
1  Science Fiction, Action, Adventure, Thriller   

                                production_companies  \
0  PolyGram Filmed Entertainment, Working Title F...   
1                                          Paramount   

                       production_countries spoken_languages    year  \
0  United Kingdom, United States of America          English  1996.0   
1                  United States of America          English  1991.0   

                                      title_year  
0                                   Fargo (1996)  
1  Star Trek VI: The Undiscovered Country (1991)  

[2 rows x 38 columns]

In [12]:
from concurrent.futures import ThreadPoolExecutor
def calculate_score_single(pair):
    u, i = pair
    print(f"Processing pair: User {u}, Item {i}")
    # Check whether the item is in the training dataset - ratings_matrix
    if i not in ratings_matrix.columns:
        return 2.5
    similarity_scores = similarity_matrix.loc[u].drop(str(u))
    normalized_ratings = ratings_matrix[i].drop(index=u)
    # Drop users that haven't rated the item
    similarity_scores.drop(index=normalized_ratings[normalized_ratings.isnull()].index.astype(str), inplace=True)
    normalized_ratings.dropna(inplace=True)
    
    # If none of the other users have rated items in common with the user in question return the baseline value
    if similarity_scores.isna().all():
        return 2.5
    
    total_score = 0
    total_weight = 0
    for v in normalized_ratings.index:
        if str(v) in ratings_matrix.columns:
            total_score += ratings_matrix[v] * similarity_scores[str(v)]
            total_weight += abs(similarity_scores[str(v)])
        else:
            pass
    if total_weight == 0:
        total_weight = 1        
    avg_user_rating = ratings_matrix.T.mean()[u]
    
    return avg_user_rating + total_score / total_weight

def calculate_scores_parallel(user_item_pairs):
    results = []

    with ThreadPoolExecutor(max_workers=12) as executor:  # Limit to 5 concurrent workers
        futures = [executor.submit(calculate_score_single, pair) for pair in user_item_pairs]
        for future in futures:
            results.append(future.result())
    
    return np.array(results)

In [13]:
test_df.shape

(9654, 38)

This is just to check the accuracy

In [14]:
test_ratings = np.array(test_df["Rating"])
user_item_pairs_list = list(zip(test_df["UserID"], test_df["MovieID"]))
pred_ratings = calculate_scores_parallel(user_item_pairs_list)

Processing pair: User 1, Item 608
Processing pair: User 2, Item 1372
Processing pair: User 3, Item 3114
Processing pair: User 5, Item 202
Processing pair: User 5, Item 1243
Processing pair: User 6, Item 3074
Processing pair: User 8, Item 345
Processing pair: User 9, Item 2692
Processing pair: User 10, Item 1203
Processing pair: User 10, Item 594
Processing pair: User 10, Item 1586
Processing pair: User 10, Item 186
Processing pair: User 11, Item 1265
Processing pair: User 13, Item 2370
Processing pair: User 15, Item 1586
Processing pair: User 15, Item 1777
Processing pair: User 17, Item 3578
Processing pair: User 17, Item 3576
Processing pair: User 18, Item 653
Processing pair: User 18, Item 1962
Processing pair: User 18, Item 2089
Processing pair: User 19, Item 2376
Processing pair: User 19, Item 2720
Processing pair: User 19, Item 1608
Processing pair: User 22, Item 3254Processing pair: User 22, Item 3120
Processing pair: User 22, Item 377

Processing pair: User 23, Item 3638
Process

In [15]:
print(np.sqrt(mean_squared_error(test_ratings, pred_ratings)))

1.0275505568290262


Getting the predicted ratings for each user based on similarity scores to another user 

In [16]:
import numpy as np

def calculate_all_ratings_vectorized(similarity_matrix, normalized_ratings_matrix):
    similarity_scores = similarity_matrix.values  # Convert similarity matrix to NumPy array
    normalized_ratings = normalized_ratings_matrix.values  # Convert normalized ratings matrix to NumPy array

    # Transpose similarity_scores to match the shape of normalized_ratings for broadcasting
    similarity_scores_transposed = similarity_scores.T

    # Calculate the dot product of similarity scores and normalized ratings
    dot_product = np.dot(similarity_scores_transposed, normalized_ratings)

    # Calculate the total weight for each user
    total_weight = np.abs(similarity_scores).sum(axis=1)

    # Calculate the average user rating
    avg_user_rating = np.nanmean(normalized_ratings, axis=1)

    # Calculate the predicted ratings for all users and items
    predicted_ratings = avg_user_rating[:, np.newaxis] + (dot_product / total_weight[:, np.newaxis])

    return predicted_ratings

# Assuming you have similarity_matrix and normalized_ratings_matrix ready
predicted_ratings = calculate_all_ratings_vectorized(similarity_matrix.copy().fillna(0), ratings_matrix.copy().fillna(0))

C:\Users\Wade\AppData\Local\Temp\ipykernel_20628\4092217865.py:20: RuntimeWarning: invalid value encountered in divide
  predicted_ratings = avg_user_rating[:, np.newaxis] + (dot_product / total_weight[:, np.newaxis])


In [18]:
predicted_rating_final = pd.DataFrame(predicted_ratings, columns = ratings_matrix.columns.values, index = ratings_matrix.index.values)
predicted_rating_final.shape

(6040, 3672)

In [19]:
# Assuming predicted_rating_final is a DataFrame where rows represent users and columns represent movies
first_10_users_ratings = predicted_rating_final.iloc[:]

# Extracting movie IDs from the column indices of the DataFrame
movie_ids = predicted_rating_final.columns

import pandas as pd

# Initialize a list to store top 5 movies for each user
top_movies_data = []

# Iterate over the first 10 users and their ratings
for user_id, user_ratings in first_10_users_ratings.iterrows():
    # Sort the predicted ratings for the user
    sorted_ratings = sorted(zip(movie_ids, user_ratings), key=lambda x: x[1], reverse=True)
    top_5_movies = sorted_ratings[:1]
    
    # Append the top 5 movies for the user to the list
    for movie_id, rating in top_5_movies:
        top_movies_data.append({'UserID': user_id, 'MovieID': movie_id, 'PredictedRating': rating})

# Create a DataFrame from the list
top_movies_df = pd.DataFrame(top_movies_data)
top_movies_df


UserID  MovieID  PredictedRating
0          1      527         0.570230
1          2     2858         1.570609
2          3      260         1.174224
3          4     2028         1.206103
4          5     2858         1.368754
...      ...      ...              ...
6035    6036     2858         3.162189
6036    6037     2858         1.661305
6037    6038     1148         0.442433
6038    6039      260         1.460411
6039    6040     2858         2.130476

[6040 rows x 3 columns]

Above is the list of users and for the movie, his predicted rating
Below we create a dictionary of index key pair value to movie title

In [20]:
# Create a dictionary to store movie ID-title pairs
movie_id_title_dict = {}

# Iterate over the DataFrame to populate the dictionary
for index, row in df.iterrows():
    movie_id = row['MovieID']
    title = row['Title']
    movie_id_title_dict[movie_id] = title

# Now movie_id_title_dict contains the mapping of movie IDs to titles
print(movie_id_title_dict)


{1193: "One Flew Over the Cuckoo's Nest (1975)", 661: 'James and the Giant Peach (1996)', 914: 'My Fair Lady (1964)', 3408: 'Erin Brockovich (2000)', 2355: "A Bug's Life (1998)", 1197: 'The Princess Bride (1987)', 1287: 'Ben-Hur (1959)', 2804: 'A Christmas Story (1983)', 594: 'Snow White and the Seven Dwarfs (1937)', 919: 'The Wizard of Oz (1939)', 595: 'Beauty and the Beast (1991)', 938: 'Gigi (1958)', 2398: 'Miracle on 34th Street (1947)', 2918: "Ferris Bueller's Day Off (1986)", 1035: 'The Sound of Music (1965)', 2791: 'Airplane! (1980)', 2687: 'Tarzan (1999)', 2018: 'Bambi (1942)', 3105: 'Awakenings (1990)', 2797: 'Big (1988)', 2321: 'Pleasantville (1998)', 720: 'Wallace & Gromit: The Best of Aardman Animation (1996)', 1270: 'Back to the Future (1985)', 527: "Schindler's List (1993)", 2340: 'Meet Joe Black (1998)', 1097: 'E.T. the Extra-Terrestrial (1982)', 1721: 'Titanic (1997)', 1545: 'Ponette (1996)', 745: 'A Close Shave (1995)', 2294: 'Antz (1998)', 3186: 'Girl, Interrupted (19

In [25]:
top_movies_df['Title'] = top_movies_df['MovieID'].map(movie_id_title_dict)
print(len(top_movies_df.Title.unique()))
# Display the DataFrame with Movie titles
top_movies_df

160


UserID  MovieID  PredictedRating  \
0          1      527         0.570230   
1          2     2858         1.570609   
2          3      260         1.174224   
3          4     2028         1.206103   
4          5     2858         1.368754   
...      ...      ...              ...   
6035    6036     2858         3.162189   
6036    6037     2858         1.661305   
6037    6038     1148         0.442433   
6038    6039      260         1.460411   
6039    6040     2858         2.130476   

                                          Title  
0                       Schindler's List (1993)  
1                        American Beauty (1999)  
2     Star Wars: Episode IV - A New Hope (1977)  
3                    Saving Private Ryan (1998)  
4                        American Beauty (1999)  
...                                         ...  
6035                     American Beauty (1999)  
6036                     American Beauty (1999)  
6037                  The Wrong Trousers (1993)  
6038  Star Wars: Episode IV - A New Hope (1977)  
6039                     American Beauty (1999)  

[6040 rows x 4 columns]

Save this output that is to be passed into the next step. We pass each user and the movie predicted
pair to a content based model to extrapolate for the current old movies.
This extrapolation predicts for this user which is his most recommended newer movies when compared
to his best old movie recommendation.

In [16]:
top_movies_df.to_csv('top1movie for each user model(useruser rating sim matrix).csv', index=False)